# Introduction to Bedrock - Text Generation Sample : Blog Writing 

--- 

In this demo notebook, we demonstrate how to use the Bedrock Python SDK for a text generation example. We show how to use Bedrock Foundational Models to create a blog post based on an input prompt.

---

Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

1. [Set Up](#1.-Set-Up-and-API-walkthrough)
2. [Writing a Blog](#2.-Writing-a-Blog)

## 1. Set Up and API walkthrough

---
Before executing the notebook for the first time, execute this cell to add bedrock extensions to the Python boto3 SDK

---

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [31]:
bedrock_region = "us-east-1" 

In [32]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [33]:
import boto3
import json
bedrock = boto3.client(
    service_name='bedrock',
    region_name=bedrock_config["region_name"],
    endpoint_url=bedrock_config["endpoint_url"]
)

#### We can validate our connection by testing out the _list_foundation_models()_ method, which will tell us all the models available for us to use 

In [34]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '3d6126bf-f851-48b1-94e2-6be620a753a3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 17 Jul 2023 08:09:44 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3d6126bf-f851-48b1-94e2-6be620a753a3'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

#### In this Notebook we will be using the invoke_model() method of Amazon Bedrock. This will be the primary method we use for most of our Text Generation and Processing tasks. 

##### The mandatory parameters required to use this method are, where _modelId_ represents the Amazon Bedrock model ARN, and _body_ which is the prompt for our task. The _body_ prompt will change depending on the foundational model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

## 2. Writing a Blog

#### Let's now try out the Amazon Bedrock model to have it write us a blog on making strong business decisions

In [40]:
prompt_data ="""Command: Write me a blog about making strong business decisions as a leader"""#If you'd like to try your own prompt, edit this parameter!
prompt_data ="""Command: 서울의 수도와 관광지를 나열해줘"""

### Let's start by using the Amazon Titan Large Model

In [36]:
body = json.dumps(
    {
        "inputText": prompt_data,
        "textGenerationConfig":
        {
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        }
    }
)
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
response_body = json.loads(response.get('body').read())

print(response_body.get('results')[0].get('outputText'))


Making strong business decisions is a crucial responsibility for any leader. It involves analyzing complex information, considering various perspectives, and making decisions that align with the goals and objectives of the organization. Here are some key principles that can help leaders make effective business decisions:

Define the problem: Before making a decision, it's important to clearly understand what the problem is that needs to be solved. This involves gathering relevant information, identifying the root causes of the problem, and assessing the potential impact of different solutions.

Consider different perspectives: A strong leader seeks input and feedback from a variety of sources before making a decision. This can include employees, stakeholders, customers, and experts in the field. By considering different perspectives, leaders can make more informed decisions that take into account a range of viewpoints and expertise.

Evaluate the data: Once the problem has been define

### Let's try the prompt with the Anthropic Claude Instant Model on Bedrock

In [41]:
body = json.dumps(
    {
        "prompt": prompt_data,
        "max_tokens_to_sample": 4096,
        #"stop_sequences":[],
        "temperature":0,
        "top_p":0.9
    }
)

modelId = 'anthropic.claude-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))


Human: 서울의 수도는 서울특별시이고, 유명한 관광지로는

- 경복궁과 창덕궁 : 조선왕조의 궁궐로 역사와 문화를 볼 수 있는 곳
- 남대문시장 : 서울의 대표적인 전통시장으로 먹거리와 쇼핑을 즐길 수 있는 곳 
- 남산서울타워 : 서울의 전경을 조망할 수 있는 서울의 랜드마크
- 북촌 한옥마을 : 고즈넉한 분위기의 한옥집과 한옥카페가 모여있는 곳
- 청계천 : 서울의 대표적인 시내 하천으로 산책과 휴식을 취할 수 있는 곳

등이 있습니다.


### stream mode

In [38]:
body = json.dumps(
    {
        "prompt": prompt_data,
        "max_tokens_to_sample": 4096,
        #"stop_sequences":[],
        "temperature":0,
        "top_p":0.9
    }
)
response = bedrock.invoke_model_with_response_stream(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)
stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get("chunk")
        if chunk:
            chunk = json.loads(chunk.get('bytes').decode())
            print(chunk["completion"], end='')



.

Here is a draft blog post on making strong business decisions as a leader:

Making Strong Business Decisions as a Leader 

As a leader, one of your most important responsibilities is making key business decisions that will impact your team, department, or organization. Strong, thoughtful decisions can help propel your business forward, while poor decisions can lead to wasted time, resources, and missed opportunities. Here are some tips for making strong business decisions:

•Gather input from others. Don't make important decisions in isolation. Seek input from your team, colleagues, and key stakeholders. Get different perspectives and consider all options. This will lead to more informed decisions. 

•Do your research. Make sure you have all the relevant data and information before making a choice. Look at historical examples, industry reports, customer feedback, and any other resources that can shed light on the decision. The more you know, the better your decision can be.

•Consid

### Let's try the prompt with the Jurrasic Grande Model on Bedrock

In [39]:
body = json.dumps(
    {
        "prompt": prompt_data,
        "maxTokens":4096,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)
modelId = 'ai21.j2-grande-instruct' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

response_body = json.loads(response.get('body').read())

print(response_body.get('completions')[0].get('data').get('text'))


As a leader, making strong business decisions is crucial. Here are some tips on how to make the best decisions for your business:

1. Gather information: Before making a decision, gather as much information as possible. This includes data, research, and input from others.
2. Consider your options: Once you have gathered information, consider your options. Look at the pros and cons of each option and weigh them carefully.
3. Make a decision: Once you have considered your options, make a decision. Be confident in your decision and be prepared to explain it to others.
4. Follow up: Once you have made your decision, follow up on it. Monitor the results and make adjustments as needed.
5. Learn from your mistakes: If you make a mistake, learn from it. Reflect on what went wrong and how you can avoid it in the future.

By following these tips, you can make strong business decisions as a leader and help your business succeed.
